<a href="https://colab.research.google.com/github/youwon000218/TABAproject_AnomalyDetect/blob/main/yolo8_extract.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ultralytics -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 780.6/780.6 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 59.8 MB/s eta 0:00:00


In [ ]:
!pip install yolov8 -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.4/953.4 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.4/88.4 kB 8.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.6/112.6 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.2/82.2 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.3/158.3 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 4.6 

In [ ]:
from collections import defaultdict
import cv2
import numpy as np
from ultralytics import YOLO
import csv
import os
from copy import deepcopy

from datetime import datetime

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


프레임마다 이미지 저장

In [ ]:
# feature extraction 함수
def feat_extraction():
    # Load the YOLOv8 model
    model = YOLO("yolov8n-pose.pt")

    # Define the standard frame size (change these values as needed)
    standard_width = 320
    standard_height = 240

    root = "/content/drive/MyDrive/taba/원천데이터/절도영상원천데이터"  # Colab의 비디오 파일 경로를 지정하세요.

    output_root = "/content/drive/MyDrive/taba/원천데이터/절도영상원천데이터/절도frameimage"  # 프레임 이미지를 저장할 폴더 경로

    folder_list = os.listdir(root)
    folder_list.sort()
    print(f"==>> folder_list: {folder_list}")

    for folder_name in folder_list:
        time_start = datetime.now()

        print(f"{folder_name} feature extracting starts")

        folder_path = os.path.join(root, folder_name)
        file_list = os.listdir(folder_path)
        file_list.sort()
        print(f"==>> file_list: {file_list}")

        for file_name in tqdm(file_list, total=len(file_list)):
            path = os.path.join(folder_path, file_name)
            cap = cv2.VideoCapture(path)

            # 프레임을 저장할 디렉토리 생성
            frame_folder = os.path.join(output_root, f"{file_name}_frames")
            if not os.path.exists(frame_folder):
                os.makedirs(frame_folder)

            frame_count = 0

            while cap.isOpened():
                success, frame = cap.read()

                if not success:
                    break

                frame_count += 1  # Increment frame count
                frame = cv2.resize(frame, (standard_width, standard_height))

                # Run YOLOv8 tracking on the frame, persisting tracks between frames
                results = model.track(frame, persist=True, verbose=False)

                # 프레임 이미지 저장
                save_path = os.path.join(frame_folder, f"frame_{frame_count:05d}.jpg")
                cv2.imwrite(save_path, frame)

            cap.release()

        time_end = datetime.now()
        total_time = time_end - time_start
        total_time = str(total_time).split(".")[0]

        print(f"{folder_name} feature extracting ended. Elapsed time: {total_time}")

# Google Colab에서 GPU 사용 확인
if 'google.colab' in str(get_ipython()):
    print("Running on Google Colab")
    import torch
    if torch.cuda.is_available():
        print("CUDA is available. Using GPU.")
    else:
        print("CUDA is not available. Using CPU.")

feat_extraction()


Running on Google Colab
CUDA is not available. Using CPU.
==>> folder_list: ['절도frameimage', '절도디렉토리']
절도frameimage feature extracting starts
==>> file_list: []


0it [00:00, ?it/s]


절도frameimage feature extracting ended. Elapsed time: 0:00:00
절도디렉토리 feature extracting starts
==>> file_list: ['C_3_12_10_BU_DYA_07-27_13-01-22_CA_RGB_DF2_F2.mp4', 'C_3_12_10_BU_DYA_07-27_13-01-25_CB_RGB_DF2_F2.mp4', 'C_3_12_10_BU_DYA_07-27_13-01-25_CC_RGB_DF2_F2.mp4', 'C_3_12_10_BU_DYA_08-10_14-54-42_CD_RGB_DF2_M2.mp4', 'C_3_12_10_BU_DYA_08-10_14-54-47_CE_RGB_DF2_M2.mp4', 'C_3_12_10_BU_DYA_08-10_14-54-47_CF_RGB_DF2_M2.mp4', 'C_3_12_10_BU_SMB_09-01_13-52-38_CA_RGB_DF2_M2.mp4', 'C_3_12_10_BU_SMB_09-01_13-52-38_CB_RGB_DF2_M2.mp4', 'C_3_12_10_BU_SMB_09-01_13-52-38_CC_RGB_DF2_M2.mp4', 'C_3_12_10_BU_SMB_09-01_13-52-38_CD_RGB_DF2_M2.mp4', 'C_3_12_10_BU_SYB_09-28_14-10-50_CA_RGB_DF2_M2.mp4', 'C_3_12_10_BU_SYB_09-28_14-10-50_CB_RGB_DF2_M2.mp4', 'C_3_12_10_BU_SYB_09-28_14-10-50_CC_RGB_DF2_M2.mp4', 'C_3_12_10_BU_SYB_09-28_14-10-50_CD_RGB_DF2_M2.mp4', 'C_3_12_11_BU_DYA_07-27_13-03-06_CA_RGB_DF2_F2.mp4', 'C_3_12_11_BU_DYA_08-10_14-59-20_CE_RGB_DF2_F2.mp4', 'C_3_12_11_BU_SMB_09-01_13-56-

  0%|          | 0/24 [00:00<?, ?it/s]

requirements: Ultralytics requirement ['lapx>=0.5.2'] not found, attempting AutoUpdate...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 21.5 MB/s eta 0:00:00

requirements: AutoUpdate success ✅ 4.9s, installed 1 package: ['lapx>=0.5.2']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect



100%|██████████| 24/24 [11:07<00:00, 27.80s/it]

절도디렉토리 feature extracting ended. Elapsed time: 0:11:07


csv로 feature저장

In [ ]:
root = "/content/drive/MyDrive/taba/원천데이터/비정상동영상"
output_root = "/content/drive/MyDrive/taba/datasetCSV/비정상csv/train_video"  # 원하는 경로로 수정하세요

header = [
    "Filename",
    "Frame",
    "ID",
    "X",
    "Y",
    "Width",
    "Height",
    "Keypoint_0",
    "Keypoint_1",
    "Keypoint_2",
    "Keypoint_3",
    "Keypoint_4",
    "Keypoint_5",
    "Keypoint_6",
    "Keypoint_7",
    "Keypoint_8",
    "Keypoint_9",
    "Keypoint_10",
    "Keypoint_11",
    "Keypoint_12",
    "Keypoint_13",
    "Keypoint_14",
    "Keypoint_15",
    "Keypoint_16",
    "Keypoint_17",
    "Keypoint_18",
    "Keypoint_19",
    "Keypoint_20",
    "Keypoint_21",
    "Keypoint_22",
    "Keypoint_23",
    "Keypoint_24",
    "Keypoint_25",
    "Keypoint_26",
    "Keypoint_27",
    "Keypoint_28",
    "Keypoint_29",
    "Keypoint_30",
    "Keypoint_31",
    "Keypoint_32",
    "Keypoint_33",
]


In [ ]:
# Load the YOLOv8 model
model = YOLO("yolov8n-pose.pt")

# Define the standard frame size (change these values as needed)
standard_width = 320
standard_height = 240

100%|██████████| 6.51M/6.51M [00:00<00:00, 86.2MB/s]


In [ ]:
folder_list = os.listdir(root)
folder_list.sort()
print(f"==>> folder_list: {folder_list}")

==>> folder_list: ['전도train', '절도train']


In [ ]:
!pip install tqdm

In [ ]:
from tqdm import tqdm

In [ ]:
for folder_name in folder_list:
    time_start = datetime.now()

    print(f"{folder_name} feature extracting starts")

    output_file_path = os.path.join(output_root, f"{folder_name}.csv")

    with open(output_file_path, "w") as c_file:
        writer = csv.writer(c_file, delimiter=",")
        writer.writerow(header)

    folder_path = root +"/" +folder_name + "/"
    file_list = os.listdir(root +"/"+folder_name)
    file_list.sort()
    print(f"==>> file_list: {file_list}")

    id_count = 0

    for file_name in tqdm(file_list, total=len(file_list)):
        path = folder_path + file_name
        cap = cv2.VideoCapture(path)

        # Loop through the video frames
        frame_count = 0

        # Store the track history
        track_history = defaultdict(lambda: [])

        while cap.isOpened():
            # Read a frame from the video
            success, frame = cap.read()
            frame_count += 1  # Increment frame count

            if success:
                frame = cv2.resize(frame, (standard_width, standard_height))

                # Run YOLOv8 tracking on the frame, persisting tracks between frames
                results = model.track(frame, persist=True, verbose=False)

                if results[0].boxes is not None:  # Check if there are results and boxes
                    if results[0].boxes.id is not None:
                        # If 'int' attribute exists (there are detections), get the track IDs
                        track_ids = results[0].boxes.id.int().cpu().tolist()

                        for i, box in zip(range(0, len(track_ids)), results[0].boxes.xywhn.cpu()):
                            keypoints = results[0].keypoints.xyn[i].cpu().numpy().flatten().tolist()
                            box_list = box.numpy().flatten().tolist()
                            if type(box_list) == "float" or type(keypoints) == "float":
                                print(f"==>> box_list: {box_list}")
                                print(f"==>> keypoints: {keypoints}")
                            box_and_keypoints = box_list + keypoints
                            track_history[track_ids[i]].append([[frame_count], deepcopy(box_and_keypoints)])
            else:
                # Break the loop if the end of the video is reached
                break

        with open(output_file_path, "a") as c_file:
            writer = csv.writer(c_file, delimiter=",")
            for key in track_history.keys():
                for f_count, b_and_k in track_history[key]:
                    row = [file_name] + f_count + [id_count + key] + b_and_k
                    writer.writerow(row)

        id_count = id_count + len(track_history.keys())
        cap.release()

    time_end = datetime.now()
    total_time = time_end - time_start
    total_time = str(total_time).split(".")[0]

    print(f"{folder_name} feature extracting ended. Elapsed time: {total_time}")

# cv2.destroyAllWindows()

전도train feature extracting starts
==>> file_list: ['C_3_7_10_BU_DYA_08-23_13-47-37_CA_RGB_DF2_M2.mp4', 'C_3_7_10_BU_DYA_08-23_13-47-40_CB_RGB_DF2_M2.mp4', 'C_3_7_10_BU_SMA_09-07_14-20-21_CA_RGB_DF2_M2.mp4', 'C_3_7_10_BU_SMA_09-07_14-20-21_CB_RGB_DF2_M2.mp4', 'C_3_7_10_BU_SMA_09-07_14-20-21_CC_RGB_DF2_M2.mp4', 'C_3_7_10_BU_SMA_09-07_14-20-21_CD_RGB_DF2_M2.mp4', 'C_3_7_10_BU_SMB_09-02_12-56-15_CA_RGB_DF2_M2.mp4', 'C_3_7_10_BU_SMB_09-02_12-56-15_CB_RGB_DF2_M2.mp4', 'C_3_7_10_BU_SMB_09-02_12-56-15_CC_RGB_DF2_M2.mp4', 'C_3_7_10_BU_SMB_09-02_12-56-15_CD_RGB_DF2_M2.mp4', 'C_3_7_10_BU_SMC_08-01_15-27-03_CD_RGB_DF2_M2.mp4', 'C_3_7_10_BU_SMC_08-01_15-27-04_CE_RGB_DF2_M2.mp4', 'C_3_7_10_BU_SMC_08-01_15-27-04_CF_RGB_DF2_M2.mp4', 'C_3_7_10_BU_SYA_09-24_11-08-55_CA_RGB_DF2_M2.mp4', 'C_3_7_10_BU_SYA_09-24_11-08-55_CB_RGB_DF2_M2.mp4', 'C_3_7_10_BU_SYA_09-24_11-08-55_CC_RGB_DF2_M2.mp4', 'C_3_7_10_BU_SYA_09-24_11-08-55_CD_RGB_DF2_M2.mp4', 'C_3_7_11_BU_DYA_07-27_12-29-26_CA_RGB_DF2_F2.mp4', 'C_3_7_11_

  0%|          | 0/464 [00:00<?, ?it/s]

requirements: Ultralytics requirement ['lapx>=0.5.2'] not found, attempting AutoUpdate...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 9.3 MB/s eta 0:00:00

requirements: AutoUpdate success ✅ 12.3s, installed 1 package: ['lapx>=0.5.2']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect



100%|██████████| 464/464 [5:32:20<00:00, 42.98s/it]


전도train feature extracting ended. Elapsed time: 5:32:20
절도train feature extracting starts
==>> file_list: ['C_3_12_10_BU_DYA_07-27_13-01-22_CA_RGB_DF2_F2.mp4', 'C_3_12_10_BU_DYA_07-27_13-01-25_CB_RGB_DF2_F2.mp4', 'C_3_12_10_BU_DYA_07-27_13-01-25_CC_RGB_DF2_F2.mp4', 'C_3_12_10_BU_DYA_08-10_14-54-42_CD_RGB_DF2_M2.mp4', 'C_3_12_10_BU_DYA_08-10_14-54-47_CE_RGB_DF2_M2.mp4', 'C_3_12_10_BU_DYA_08-10_14-54-47_CF_RGB_DF2_M2.mp4', 'C_3_12_10_BU_SMB_09-01_13-52-38_CA_RGB_DF2_M2.mp4', 'C_3_12_10_BU_SMB_09-01_13-52-38_CB_RGB_DF2_M2.mp4', 'C_3_12_10_BU_SMB_09-01_13-52-38_CC_RGB_DF2_M2.mp4', 'C_3_12_10_BU_SMB_09-01_13-52-38_CD_RGB_DF2_M2.mp4', 'C_3_12_10_BU_SYB_09-28_14-10-50_CA_RGB_DF2_M2.mp4', 'C_3_12_10_BU_SYB_09-28_14-10-50_CB_RGB_DF2_M2.mp4', 'C_3_12_10_BU_SYB_09-28_14-10-50_CC_RGB_DF2_M2.mp4', 'C_3_12_10_BU_SYB_09-28_14-10-50_CD_RGB_DF2_M2.mp4', 'C_3_12_11_BU_DYA_07-27_13-03-06_CA_RGB_DF2_F2.mp4', 'C_3_12_11_BU_DYA_07-27_13-03-09_CC_RGB_DF2_F2.mp4', 'C_3_12_11_BU_DYA_08-10_14-59-14_CD_RGB

  1%|          | 7/600 [05:29<7:44:54, 47.04s/it]


KeyboardInterrupt: 

In [ ]:
import pandas as pd

# CSV 파일 경로
csv_file_path = "/content/drive/MyDrive/taba/datasetCSV/비정상csv/train_video/절도train.csv"

# CSV 파일을 DataFrame으로 읽기
df = pd.read_csv(csv_file_path)
df.head()

,Filename,Frame,ID,X,Y,Width,Height,Keypoint_0,Keypoint_1,Keypoint_2,...,Keypoint_24,Keypoint_25,Keypoint_26,Keypoint_27,Keypoint_28,Keypoint_29,Keypoint_30,Keypoint_31,Keypoint_32,Keypoint_33
0,C_3_12_10_BU_DYA_07-27_13-01-22_CA_RGB_DF2_F2.mp4,10,827,0.489624,0.269538,0.068141,0.355832,0.484616,0.138559,0.490313,...,0.477813,0.284056,0.501813,0.357834,0.473611,0.361816,0.502281,0.422351,0.466512,0.427871
1,C_3_12_10_BU_DYA_07-27_13-01-22_CA_RGB_DF2_F2.mp4,11,827,0.472495,0.278553,0.070988,0.360915,0.469000,0.144973,0.472167,...,0.457613,0.290382,0.476794,0.359930,0.462512,0.362543,0.472239,0.428739,0.461845,0.434743
2,C_3_12_10_BU_DYA_07-27_13-01-22_CA_RGB_DF2_F2.mp4,12,827,0.460539,0.300907,0.079102,0.388304,0.469401,0.156068,0.470660,...,0.451392,0.312118,0.467668,0.390724,0.460580,0.391427,0.463293,0.467144,0.462968,0.470597
3,C_3_12_10_BU_DYA_07-27_13-01-22_CA_RGB_DF2_F2.mp4,13,827,0.455871,0.310842,0.083950,0.403077,0.469199,0.153655,0.471828,...,0.442740,0.319618,0.468097,0.400696,0.444740,0.401803,0.461607,0.476752,0.442642,0.479648
4,C_3_12_10_BU_DYA_07-27_13-01-22_CA_RGB_DF2_F2.mp4,14,827,0.450323,0.331396,0.085604,0.436700,0.465528,0.158185,0.000000,...,0.444468,0.350736,0.468221,0.437126,0.447811,0.442065,0.459423,0.517988,0.442188,0.526593


In [ ]:
df['Filename'].unique().shape

(7,)

In [ ]:
df

,Filename,Frame,ID,X,Y,Width,Height,Keypoint_0,Keypoint_1,Keypoint_2,...,Keypoint_24,Keypoint_25,Keypoint_26,Keypoint_27,Keypoint_28,Keypoint_29,Keypoint_30,Keypoint_31,Keypoint_32,Keypoint_33
0,C_3_12_10_BU_DYA_07-27_13-01-22_CA_RGB_DF2_F2.mp4,10,827,0.489624,0.269538,0.068141,0.355832,0.484616,0.138559,0.490313,...,0.477813,0.284056,0.501813,0.357834,0.473611,0.361816,0.502281,0.422351,0.466512,0.427871
1,C_3_12_10_BU_DYA_07-27_13-01-22_CA_RGB_DF2_F2.mp4,11,827,0.472495,0.278553,0.070988,0.360915,0.469000,0.144973,0.472167,...,0.457613,0.290382,0.476794,0.359930,0.462512,0.362543,0.472239,0.428739,0.461845,0.434743
2,C_3_12_10_BU_DYA_07-27_13-01-22_CA_RGB_DF2_F2.mp4,12,827,0.460539,0.300907,0.079102,0.388304,0.469401,0.156068,0.470660,...,0.451392,0.312118,0.467668,0.390724,0.460580,0.391427,0.463293,0.467144,0.462968,0.470597
3,C_3_12_10_BU_DYA_07-27_13-01-22_CA_RGB_DF2_F2.mp4,13,827,0.455871,0.310842,0.083950,0.403077,0.469199,0.153655,0.471828,...,0.442740,0.319618,0.468097,0.400696,0.444740,0.401803,0.461607,0.476752,0.442642,0.479648
4,C_3_12_10_BU_DYA_07-27_13-01-22_CA_RGB_DF2_F2.mp4,14,827,0.450323,0.331396,0.085604,0.436700,0.465528,0.158185,0.000000,...,0.444468,0.350736,0.468221,0.437126,0.447811,0.442065,0.459423,0.517988,0.442188,0.526593
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1151,C_3_12_10_BU_SMB_09-01_13-52-38_CA_RGB_DF2_M2.mp4,166,838,0.648257,0.455283,0.139293,0.476009,0.706668,0.294961,0.715872,...,0.662524,0.474765,0.651385,0.533357,0.668393,0.571004,0.623613,0.596407,0.669848,0.667533
1152,C_3_12_10_BU_SMB_09-01_13-52-38_CA_RGB_DF2_M2.mp4,167,838,0.690901,0.492977,0.157536,0.501501,0.764941,0.323202,0.774128,...,0.724010,0.527524,0.706064,0.590050,0.734560,0.632093,0.649108,0.627152,0.711704,0.704940
1153,C_3_12_10_BU_SMB_09-01_13-52-38_CA_RGB_DF2_M2.mp4,168,838,0.757600,0.515592,0.177993,0.505986,0.865425,0.354515,0.870358,...,0.788414,0.548447,0.764368,0.623658,0.806251,0.652176,0.715007,0.697207,0.794290,0.744246
1154,C_3_12_10_BU_SMB_09-01_13-52-38_CA_RGB_DF2_M2.mp4,169,838,0.828528,0.568615,0.239081,0.553231,0.000000,0.000000,0.000000,...,0.880064,0.620627,0.836586,0.703723,0.886512,0.758354,0.767661,0.717649,0.850268,0.811980


In [ ]:
# Group by 'Filename' and save each group to a separate CSV file
grouped = df.groupby('Filename')

# 각 Filename 별로 CSV 파일을 저장
for filename, group in grouped:
    # Removing special characters from the filename to make it suitable for saving as a CSV file
    clean_filename = "".join(x for x in filename if (x.isalnum() or x in "._-"))
    csv_filename = f"/content/drive/MyDrive/taba/datasetCSV/비정상csv/train_video/{clean_filename}.csv"
    group.to_csv(csv_filename, index=False)

print("CSV files have been created successfully.")

CSV files have been created successfully.


In [ ]:
import os

# 디렉토리 경로 설정
directory_path = '/content/drive/MyDrive/taba/datasetCSV/정상csv/train_video'

# 디렉토리 내 파일 목록 확인
files = os.listdir(directory_path)

# CSV 파일 개수 확인
csv_files = [file for file in files if file.endswith('.csv')]
csv_file_count = len(csv_files)

print(f"CSV 파일 개수: {csv_file_count}")

CSV 파일 개수: 350


정상행동 feature추출

In [ ]:
root = "/content/drive/MyDrive/taba/원천데이터/정상행동원천데이터/"
output_root = "/content/drive/MyDrive/taba/유원/학습데이터_yolo8/"  # 원하는 경로로 수정하세요

header = [
    "Filename",
    "Frame",
    "ID",
    "X",
    "Y",
    "Width",
    "Height",
    "Keypoint_0",
    "Keypoint_1",
    "Keypoint_2",
    "Keypoint_3",
    "Keypoint_4",
    "Keypoint_5",
    "Keypoint_6",
    "Keypoint_7",
    "Keypoint_8",
    "Keypoint_9",
    "Keypoint_10",
    "Keypoint_11",
    "Keypoint_12",
    "Keypoint_13",
    "Keypoint_14",
    "Keypoint_15",
    "Keypoint_16",
    "Keypoint_17",
    "Keypoint_18",
    "Keypoint_19",
    "Keypoint_20",
    "Keypoint_21",
    "Keypoint_22",
    "Keypoint_23",
    "Keypoint_24",
    "Keypoint_25",
    "Keypoint_26",
    "Keypoint_27",
    "Keypoint_28",
    "Keypoint_29",
    "Keypoint_30",
    "Keypoint_31",
    "Keypoint_32",
    "Keypoint_33",
]


In [ ]:
# Load the YOLOv8 model
model = YOLO("yolov8n-pose.pt")

# Define the standard frame size (change these values as needed)
standard_width = 320
standard_height = 240

100%|██████████| 6.51M/6.51M [00:00<00:00, 168MB/s]


In [ ]:
folder_list = os.listdir(root)
folder_list.sort()
print(f"==>> folder_list: {folder_list}")

==>> folder_list: ['정상디렉토리']


In [ ]:
!pip install tqdm

In [ ]:
from tqdm import tqdm

In [ ]:
for folder_name in folder_list:
    time_start = datetime.now()

    print(f"{folder_name} feature extracting starts")

    output_file_path = os.path.join(output_root, f"{folder_name}.csv")

    with open(output_file_path, "w") as c_file:
        writer = csv.writer(c_file, delimiter=",")
        writer.writerow(header)

    folder_path = root + folder_name + "/"
    file_list = os.listdir(root + folder_name)
    file_list.sort()
    print(f"==>> file_list: {file_list}")

    id_count = 0

    for file_name in tqdm(file_list, total=len(file_list)):
        path = folder_path + file_name
        cap = cv2.VideoCapture(path)

        # Loop through the video frames
        frame_count = 0

        # Store the track history
        track_history = defaultdict(lambda: [])

        while cap.isOpened():
            # Read a frame from the video
            success, frame = cap.read()
            frame_count += 1  # Increment frame count

            if success:
                frame = cv2.resize(frame, (standard_width, standard_height))

                # Run YOLOv8 tracking on the frame, persisting tracks between frames
                results = model.track(frame, persist=True, verbose=False)

                if results[0].boxes is not None:  # Check if there are results and boxes
                    if results[0].boxes.id is not None:
                        # If 'int' attribute exists (there are detections), get the track IDs
                        track_ids = results[0].boxes.id.int().cpu().tolist()

                        for i, box in zip(range(0, len(track_ids)), results[0].boxes.xywhn.cpu()):
                            keypoints = results[0].keypoints.xyn[i].cpu().numpy().flatten().tolist()
                            box_list = box.numpy().flatten().tolist()
                            if type(box_list) == "float" or type(keypoints) == "float":
                                print(f"==>> box_list: {box_list}")
                                print(f"==>> keypoints: {keypoints}")
                            box_and_keypoints = box_list + keypoints
                            track_history[track_ids[i]].append([[frame_count], deepcopy(box_and_keypoints)])
            else:
                # Break the loop if the end of the video is reached
                break

        with open(output_file_path, "a") as c_file:
            writer = csv.writer(c_file, delimiter=",")
            for key in track_history.keys():
                for f_count, b_and_k in track_history[key]:
                    row = [file_name] + f_count + [id_count + key] + b_and_k
                    writer.writerow(row)

        id_count = id_count + len(track_history.keys())
        cap.release()

    time_end = datetime.now()
    total_time = time_end - time_start
    total_time = str(total_time).split(".")[0]

    print(f"{folder_name} feature extracting ended. Elapsed time: {total_time}")

# cv2.destroyAllWindows()

정상디렉토리 feature extracting starts
==>> file_list: ['C_2_4_47_BU_SMC_08-20_14-36-16_CC_RGB_DF1_M4.mp4', 'C_2_4_48_BU_DYA_08-24_17-19-00_CA_RGB_DF1_M2_F2.mp4', 'C_2_4_48_BU_DYA_08-24_17-19-00_CB_RGB_DF1_M2_F2.mp4', 'C_2_4_48_BU_SMB_10-16_13-47-06_CA_RGB_DF1_F3_F3.mp4', 'C_2_4_48_BU_SMB_10-16_13-47-06_CB_RGB_DF1_F3_F3.mp4', 'C_2_4_5_BU_DYB_10-10_10-04-27_CC_RGB_DF1_F1_F1.mp4']


  0%|          | 0/6 [00:00<?, ?it/s]

requirements: Ultralytics requirement ['lapx>=0.5.2'] not found, attempting AutoUpdate...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 14.7 MB/s eta 0:00:00

requirements: AutoUpdate success ✅ 7.1s, installed 1 package: ['lapx>=0.5.2']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect



100%|██████████| 6/6 [13:09<00:00, 131.56s/it]

정상디렉토리 feature extracting ended. Elapsed time: 0:13:09


In [ ]:
import pandas as pd

# CSV 파일 경로
csv_file_path = "/content/drive/MyDrive/taba/유원/학습데이터_yolo8/정상디렉토리.csv"

# CSV 파일을 DataFrame으로 읽기
df = pd.read_csv(csv_file_path)
df

,Filename,Frame,ID,X,Y,Width,Height,Keypoint_0,Keypoint_1,Keypoint_2,...,Keypoint_24,Keypoint_25,Keypoint_26,Keypoint_27,Keypoint_28,Keypoint_29,Keypoint_30,Keypoint_31,Keypoint_32,Keypoint_33
0,C_2_4_47_BU_SMC_08-20_14-36-16_CC_RGB_DF1_M4.mp4,17,1,0.127809,0.690421,0.255619,0.619157,0.000000,0.00000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,C_2_4_47_BU_SMC_08-20_14-36-16_CC_RGB_DF1_M4.mp4,18,1,0.157616,0.681744,0.283377,0.636465,0.000000,0.00000,0.0,...,0.202913,0.980918,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,C_2_4_47_BU_SMC_08-20_14-36-16_CC_RGB_DF1_M4.mp4,19,1,0.214983,0.667670,0.269466,0.664616,0.000000,0.00000,0.0,...,0.264057,0.932031,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,C_2_4_47_BU_SMC_08-20_14-36-16_CC_RGB_DF1_M4.mp4,20,1,0.258317,0.654149,0.262816,0.691664,0.000000,0.00000,0.0,...,0.300452,0.865637,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,C_2_4_47_BU_SMC_08-20_14-36-16_CC_RGB_DF1_M4.mp4,21,1,0.295009,0.646671,0.253518,0.705948,0.000000,0.00000,0.0,...,0.309065,0.824839,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3795,C_2_4_5_BU_DYB_10-10_10-04-27_CC_RGB_DF1_F1_F1...,562,36,0.875316,0.643167,0.212795,0.420768,0.970173,0.50563,0.0,...,0.885343,0.668392,0.914567,0.744660,0.844365,0.719113,0.910876,0.812647,0.791207,0.769386
3796,C_2_4_5_BU_DYB_10-10_10-04-27_CC_RGB_DF1_F1_F1...,563,36,0.895184,0.645874,0.209631,0.422166,0.000000,0.00000,0.0,...,0.928913,0.673755,0.939966,0.751352,0.890139,0.732422,0.923094,0.816132,0.831318,0.781455
3797,C_2_4_5_BU_DYB_10-10_10-04-27_CC_RGB_DF1_F1_F1...,564,36,0.919048,0.648019,0.161904,0.419751,0.000000,0.00000,0.0,...,0.959845,0.659600,0.924082,0.732156,0.947727,0.752427,0.903832,0.795514,0.924645,0.825707
3798,C_2_4_5_BU_DYB_10-10_10-04-27_CC_RGB_DF1_F1_F1...,565,36,0.940925,0.652641,0.118150,0.415729,0.000000,0.00000,0.0,...,0.967524,0.643273,0.951920,0.731687,0.957051,0.738033,0.920299,0.817791,0.923084,0.827430
